In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import requests
from urllib.parse import urlencode

### Function Description

**Retention rate** shows how many clients return to the application for a certain period after installation.

To use the function, you need to set the start date and the end date of the period.

**Requirements**
1. The start date must be greater than the end date of the period and is indicated in quotation marks.
2. The function is configured for the number of days, if you need to count by months, then you need to change the date format and bring it to the beginning of the month.

**Method of operation**
1. Accepts two entry dates (the beginning and the end of the period).
2. Reads two links (registration dates and event dates), converts the date format, makes a filter
3. Prepares both tables for merging:
* ***Registration table***: For each registration record, a list of all dates from the beginning to the end of the period is created
* ***Table with events***: mark each row by symbol (1) so that when combining it was clear on what date the action was (this approach was chosen so that not only the date of the event, but also any chosen action could be selected for the target action, for example, the date where the price is indicated, and put 1 next to the columns with the price, then counting retention on them)
4. Tables are combined by columns uid and date of events (we get a table where for each user there is a registration date, all possible dates from the list, the units affixed to the dates where the action was)
5. We calculate retention through a pivot table, where for values we take the average of the columns with bullits (there was an action and there was no action), or we take the amount of the column with actions and count the percentage from the first day, where 100% of users)

In [3]:
# read data from yandex disk
def ya_disk_download(url):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    final_url = base_url + urlencode(dict(public_key=url)) 
    response = requests.get(final_url) 
    download_url = response.json()['href'] 
 
    download_response = requests.get(download_url) 
    df = pd.read_csv(download_url, sep=';')
    return df

In [4]:
registration_url = 'https://disk.yandex.ru/d/PczS9KCTyxCMvw'
actions_url = 'https://disk.yandex.ru/d/noL8tO5a2q-_yg'

In [5]:
registration = ya_disk_download(registration_url)
actions = ya_disk_download(actions_url)

In [13]:
# table with reg date and user id
registration.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


In [14]:
# table with action date nd user id
actions.head()

,days,uid,action
0,1998-11-18,1,1
1,1999-07-22,2,1
2,1999-07-25,2,1
3,1999-07-31,2,1
4,1999-08-05,2,1


In [15]:
def retention(reg_data, auth_data, a, b):
    '''
    Retention calculation function for the specified period
    Input data:
        reg_data - dataframe with the date of user registration
        auth_data - dataframe with targeted user actions
        a - start of the period
        b - end of the period

    Output data:
        column with total base in group
        retention - table with retention by day
    '''
    reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'], unit='s')
    reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'].dt.date, format='%Y-%m-%d')
    
    auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'], unit='s')
    auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'].dt.date, format='%Y-%m-%d')
    auth_data['action'] = 1  
    auth_data.columns = ['days', 'uid', 'action']
    
    # filter for dates
    reg_data = reg_data.query('reg_ts >= @a and reg_ts <= @b')
    auth_data = auth_data.query('days >= @a and days <= @b')
    
    # for each user make a list of all dates in the periodе
    days = pd.DataFrame(pd.date_range(a, b))
    days.columns = ['days']
    
    reg_data['key'] = 1
    days['key'] = 1

    reg_data = reg_data.merge(days, how='outer', on='key')
    reg_data = reg_data.query('days >= reg_ts')
    
    df = reg_data.merge(auth_data, on=['uid', 'days'], how='left')
    df['activity'] = (df['action'] > 0) * 1
    
    # creating a column with cohorts (number of days after registration) and retention
    df['lifetime'] = ((df['days'] - df['reg_ts']) / np.timedelta64(1, 'D')).round().astype(int)
    
    retention = df.pivot_table(index='reg_ts', columns='lifetime', values='activity', aggfunc='sum')
    cohort = retention.iloc[:, 0]  # creating a separate column with the number of people in the cohort
    retention = retention.divide(cohort, axis=0)
    retention[0] = cohort.astype('int32')
    
    # plotting
    retention.index = retention.index.astype('string').str.replace(' .*', '')
    cm = sns.light_palette("#a275ac", as_cmap=True)
    df_style = (
        retention
        .style
        .set_caption('Cohorts: User Retention')
        .background_gradient(cmap=cm, subset=retention.select_dtypes(float).columns)   
        .highlight_null('white')
        .format("{:.0%}", na_rep="", subset=retention.select_dtypes(float).columns))
    return (df_style)

In [7]:
retention(registration, actions, "2016-12-07", "2016-12-16")

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


lifetime,0,1,2,3,4,5,6,7,8,9
reg_ts,,,,,,,,,,
2016-12-07,168,2%,4%,6%,9%,8%,7%,5%,5%,6%
2016-12-08,169,1%,5%,4%,5%,7%,12%,4%,4%,
2016-12-09,168,3%,2%,6%,3%,4%,8%,8%,,
2016-12-10,170,1%,5%,2%,5%,7%,4%,,,
2016-12-11,169,3%,7%,7%,3%,6%,,,,
2016-12-12,170,4%,5%,4%,4%,,,,,
2016-12-13,170,3%,2%,4%,,,,,,
2016-12-14,170,0%,4%,,,,,,,
2016-12-15,170,1%,,,,,,,,


In [16]:
print(retention.__doc__)


    Retention calculation function for the specified period
    Input data:
        reg_data - dataframe with the date of user registration
        auth_data - dataframe with targeted user actions
        a - start of the period
        b - end of the period

    Output data:
        column with total base in group
        retention - table with retention by day
    
